<a href="https://colab.research.google.com/github/bhowad-akash/Machine-Learning/blob/main/GPT_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222323 sha256=c5d5ae6680fedf4b995d40e94c417526e32ea9ea4fa987b4d87312d1e5124ba4
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


In [2]:
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pylzma
import pandas as pd
import torch
import pickle


'''CPU are sequential in nature, processing the data one aftre the other.
Whereas, GPUs can process the multiple data parallely.'''

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

cuda


In [3]:
!python --version

Python 3.10.12


In [4]:
from google.colab import files
uploaded = files.upload()

Saving wizard_of_oz.txt to wizard_of_oz.txt


In [5]:
import os
os.listdir()

['.config', 'wizard_of_oz.txt', 'sample_data']

In [6]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [7]:
print(text[:200])

﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [8]:
#Printing all the characters in the text piece
chars = sorted(list(set(text)))
print(chars)
vocab_size = len(chars)

print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [9]:
#Building a Tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

print(string_to_int)
print(int_to_string)

'''Encode is a lambda function that takes a string s as input and returns integer values from the 'string_to_int' dictionary,
whereas decode is a lambda function that takes integers as a input and returns characters, with join forming the resulting string'''

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31:

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)    #encode all the words present in the text piece
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [11]:
#Dividing the data into Training and Validation
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]
print(len(data), len(train_data), len(valid_data))

232309 185847 46462


In [12]:
#Function to generate batches of data for training or validation
def get_batch(split):
  data = train_data if split == 'train' else valid_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) #Randomly select starting indices for the batch
  x = torch.stack([data[i:i+block_size] for i in ix])       #Create input sequences by stacking slices of data starting at each index in 'ix'
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])   #Create input sequences by stacking slices of data starting at each index in 'ix'
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 61, 58,  ..., 68, 74, 72],
        [69,  1, 60,  ..., 57,  1, 59],
        [73, 61, 58,  ...,  1, 73, 61],
        ...,
        [57,  1, 54,  ..., 61, 58,  1],
        [72, 61, 62,  ..., 57, 68, 67],
        [ 0,  0, 44,  ...,  1, 61, 54]], device='cuda:0')
targets:
tensor([[61, 58, 71,  ..., 74, 72, 58],
        [ 1, 60, 54,  ...,  1, 59, 71],
        [61, 58, 66,  ..., 73, 61, 58],
        ...,
        [ 1, 54, 67,  ..., 58,  1, 47],
        [61, 62, 67,  ..., 68, 67, 58],
        [ 0, 44, 61,  ..., 61, 54, 74]], device='cuda:0')


In [13]:
'''Temporarily setting all the requires_grad flags to false, to disable gradient computation,
which is not needed during the evaluation phase and can save memory and computation.'''
@torch.no_grad()

#Defining the function to estimate loss, evaluating model's performance periodically during training
def estimate_loss():
  out = {}   #To store the average loss calculated for each data split ('train' and 'val')
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)    #Initializes a tensor of zeros with a size equal to eval_iters
    for k in range(eval_iters):
      X, Y = get_batch(split)           #Function to fetch a batch of input data (X) and corresponding target data (Y) from either the training set or validation set.
      logits, loss = model(X, Y)        #Feeding batch of data (X) into the model and computing the output (logits) and the loss (loss)
      losses[k] = loss.item()           #Extracts the scalar value of the loss using and assigns it to the k-th position in the losses tensor
    out[split] = losses.mean()          #Calculates the mean of all values in the 'losses'and stores it in the out dictionary
  model.train()                          #Switching model back to the training mode
  return out

In [14]:
'''The Head class for Scaled Dot Product Attention'''
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)     #Key K
        self.query = nn.Linear(n_embd, head_size, bias=False)   #Query Q
        self.value = nn.Linear(n_embd, head_size, bias=False)   #Value V
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))    #Masking

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


'''Multiple Head Attention class for the Multiple Head Attention BLock'''

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])     #Heads in parallel for each head size
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # Concatenation (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out


'''Feed Forward class for the Feed FOrward Network'''

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),     #Linear
            nn.ReLU(),                         #Relu
            nn.Linear(4 * n_embd, n_embd),     #Linear
            nn.Dropout(dropout),               #Dropout to prevent overfitting
        )

    def forward(self, x):
        return self.net(x)



''' Block class for the Decoder blocks between the input and output blocks'''

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)     #Self Attention
        x = self.ln1(x + y)   #Add + Normalisation
        y = self.ffwd(x)      #Forward Network
        x = self.ln2(x + y)   #Add + Normalisation
        return x



'''The below class is defined for the tOkenization, I/p and positional embeddings, layer normalisation and linear transformation'''

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):   #vocab_size represents the number of unique tokens that model can handle
        super().__init__()            #Ensures the initialization logic of the base class (nn.Module) is execute
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)   #initializes an nn.Embedding layer and assigning it to the token_embedding_table attribute
        self.position_embedding_table = nn.Embedding(block_size, n_embd)   #Initialize positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])    #Decoder blocks added sequetially
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm before data is fed to the Linear Transfomration BLocks
        self.lm_head = nn.Linear(n_embd, vocab_size)     #LInear Transfomration Block

        self.apply(self._init_weights)

    #Initializing the weights of the model using Std. deviation 0.02.
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):      #Defining forward method through which data passes through the network
        B, T = index.shape

        #TOkenization, I/p and positional embeddings, layer normalisation and linear transformation
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        '''Conditional Loss Computation: If target is None, model is being used for inference.
        ELse Model is used for training'''

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape          #Unpacks the shape of Logits into three variabless. B - Batch Size, T - Sequence Length, C - Size of Embeddings
            logits = logits.view(B*T, C)    #Reshaping from 3D to 2D. Loss function expects 2D inputs.
            targets = targets.view(B*T)     #Flattens targets to match the first dimension of logits, for computing loss at each position.
            loss = F.cross_entropy(logits, targets)    #Computes the cross-entropy loss between logits (predictions) and targets (actual outcomes).

        return logits, loss

    '''Defining method to generate text or sequences of tokens, which takes two parameters.
    index is expected to be a tensor of shape (B, T).
    max_new_tokens is the maximum number of new tokens to be generated per sequence in the batch'''

    def generate(self, index, max_new_tokens):
      for _ in range(max_new_tokens):             #Iterates max_new_tokens times, generating one new token per iteration
        logits, loss = self.forward(index)        #Calls the forward method of the model to obtain predictions (logits) based on the current index (context)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [15]:
'''Creating the Model Instance. 'vocab_size' specifies the size of vocabulary
that model will use. This parameter impacts the size of the embedding layer within the model.'''

model = GPTLanguageModel(vocab_size)
m = model.to(device)    # Moves the model instance to the specified device, which could be a GPU or CPU.

In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

0
step: 0, train loss: 3.142, val loss: 3.155
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 3.144, val loss: 3.162
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
step: 200, train loss: 3.141, val loss: 3.154
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
24

In [16]:
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)